In [43]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd

# Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
tickers = ['WALMEX']

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2005-01-01'
end_date = '2018-09-12'

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
panel_data = data.DataReader('WALMEX.MX', 'yahoo', start_date, end_date)

In [44]:
panel_data.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2018-09-06,52.810001,52.169998,52.459999,52.680000,10715715.0,52.680000
2018-09-07,52.599998,51.619999,52.520000,52.250000,8231533.0,52.250000
2018-09-10,52.900002,51.860001,52.419998,52.759998,7376673.0,52.759998
2018-09-11,53.549999,52.320000,52.400002,53.259998,15278685.0,53.259998
2018-09-12,53.330002,52.360001,53.130001,52.669998,15867470.0,52.669998
